# Import libraries

In [8]:
import os
import pandas as pd

# Fetching the data

In [12]:
EGR1_PATH = "./data/GA"
def load_egr1_data(file_path = EGR1_PATH):
    return pd.read_csv(file_path, sep="\t")

In [13]:
test = load_egr1_data()
test.head()

,sample,TCGA-AB-2880-03,TCGA-AB-2819-03,TCGA-AB-2875-03,TCGA-AB-2808-03,TCGA-AB-2848-03,TCGA-AB-2841-03,TCGA-AB-2910-03,TCGA-AB-2920-03,TCGA-AB-2986-03,...,TCGA-AB-2946-03,TCGA-AB-2999-03,TCGA-AB-2938-03,TCGA-AB-2913-03,TCGA-AB-2810-03,TCGA-AB-2924-03,TCGA-AB-2937-03,TCGA-AB-2966-03,TCGA-AB-3012-03,TCGA-AB-2954-03
0,UBE2Q1,4.5844,4.9574,4.6304,4.6482,4.5685,4.6315,4.8461,4.9947,5.1369,...,4.7872,4.6204,4.9043,4.5906,5.1216,4.7572,5.0893,5.0708,4.5537,4.6225
1,HIF3A,0.0039,0.7462,0.6472,0.0305,0.0206,0.0000,0.0068,0.0105,0.0062,...,0.0000,0.0377,0.1892,0.0342,0.0230,0.0158,3.3688,0.0541,0.0171,0.0126
2,RNF17,0.0338,0.0201,0.0000,0.0260,0.0263,0.0205,0.0178,0.0119,0.0000,...,0.0259,0.0345,0.0053,0.0404,0.0335,0.0000,0.0240,0.0334,0.0009,0.0014
3,RNF10,5.9150,5.5003,5.6430,5.7971,6.0917,6.3663,5.6803,6.0212,5.5240,...,5.5981,5.6837,5.7704,5.6478,6.4578,5.6424,5.9336,5.4772,5.9701,5.3745
4,RNF11,4.1399,4.7407,4.1058,4.8867,4.0516,4.7545,4.2329,3.7790,4.6918,...,4.2740,4.6785,4.7170,4.5241,5.2825,4.2691,5.0298,5.0714,5.0994,3.9462
